In [46]:
from pymongo import MongoClient
import csv
import time
import pandas as pd

In [47]:
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient.ProjectIF29
collection = db['Tweets']

### Generer le nombre de followers

In [19]:
data = ["ID", "Nbre Followers", "Nbre Profils suivi", "Ratio"]

with open('labels.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(data)
    i = 0
    for doc in collection.find():
        i = i+1
        donnees = [i, doc['user']['followers_count'], doc['user']['friends_count'],
                   round(doc['user']['followers_count']/doc['user']['friends_count'], 2)]
        writer.writerow(donnees)

ZeroDivisionError: division by zero

In [28]:
import pandas as pd
col = db['tweet']
cursor = col.aggregate(
    [
        {"$group" : {
            "_id" : "$user.id", 
            "friends_count" : {"$last" : "$user.friends_count"},
            "listed_count" : {"$last" : "$user.listed_count"},
            "favourites_count" : {"$last" : "$user.favourites_count"},
            "statuses_count" : {"$last" : "$user.statuses_count"},
            "followers_count" : {"$last" : "$user.followers_count"},
            "account_created_time" : {"$last" : "$user.created_at"},
            "tweet_created_time" : {"$first": "$created_at"}
        }
        }
    ]
)
entetes = ['_id', 'friends_count', 'listed_count', 'favourites_count', 
           'statuses_count', 'followers_count', 'account_created_time', 'tweet_created_time']
#Enregistrer le curseur dans un fichier  csv
with open('nombre.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=entetes)
    writer.writeheader()
    writer.writerows(cursor)

### Recuperation des textes des tweets

In [59]:
col = db['tweet']
cursor1 = col.aggregate(
    [
        {"$group" : {
            "_id" : "$user.id", 
            "Les_tweets" : {"$push": "$text"}
        }
        }
    ]
)
entetes = ['_id', 'Les_tweets']
with open('Les_tweets.csv', 'w', newline ='', encoding = 'utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=entetes)
    writer.writeheader()
    writer.writerows(cursor1)

In [53]:
def frequencePub(row):
    #la conversion en heure du tweet
    t0 = time.mktime( time.strptime(row["tweet_created_time"],
                               "%a %b %d %H:%M:%S +0000 %Y"))
    t1 = time.mktime(time.strptime(row["account_created_time"], '%a %b %d %H:%M:%S %Y'))
    return row["statuses_count"]/(t_0 - t_1)*100
data = pd.read_csv('nombre.csv')
data['frequence_statues'] = data.apply(ratio)

KeyError: 'tweet_created_time'

### ratio nombre de followers et nombre profils suivi

In [54]:
def division(df):
    if df['friends_count'] !=0:
        return df['followers_count']/df['friends_count']
    else:
        return 0
data['ratio'] = data.apply(division, axis=1)

In [55]:
data['ratio']==0

0       False
1       False
2       False
3       False
4       False
        ...  
1698    False
1699    False
1700    False
1701    False
1702    False
Name: ratio, Length: 1703, dtype: bool

### Recherche de # et @

In [60]:
def hashtag(df):
    diese = df['Les_tweets'].count('#')
    arobaz = df['Les_tweets'].count('@')
    visibile= (diese*11.6 + arobaz*11.4)/140
    return visibile
df = pd.read_csv('Les_tweets.csv')
df['visibilite'] = df.apply(hashtag, axis=1)

In [61]:
df

,_id,Les_tweets,visibilite
0,7.712831e+08,['Finally. #WorldCup'],0.082857
1,3.361040e+08,"[""RT @CadreFloyd: Some of the 'men' who'll be ...",0.164286
2,4.165152e+07,['RT @CBSThisMorning: 2026 #WorldCup announcem...,0.245714
3,9.067093e+17,['RT @billboard: Shawn Mendes drops new versio...,0.164286
4,5.628488e+07,['Who you got in the #WorldCup@@Briana_Lee_xx ...,0.245714
...,...,...,...
1698,1.000244e+18,['RT @kialatino: Cuándo sabes que la #CopaMund...,0.330000
1699,1.265995e+09,['So funny. #WorldCup https://t.co/g8bTp0CZwZ'],0.082857
1700,1.941272e+09,['RT @BBST15: Skuad Germany🇩🇪 untuk Piala Duni...,0.247143
1701,2.938594e+09,['RT @SaudiNT: هز الشباك.. جمهورك وراك! 😍💚\n\n...,0.330000
